In [1]:
# importing the necessary libraries
import glob
import os
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch


In [2]:
# defining the paths
shapefile_path = "/shared_space/BrazilSPEI/Maranhao_boundaries_EPSG/maranhao_boundary.shp"
data_dir = "/shared_space/BrazilSPEI/MODIS/Reclass_Data"
output_gif = "maranhao_multiclass_animation.gif"

fps = 2
dpi = 150
interval = 600

In [4]:
# loading the Maranhao file
print("Loading Maranhão shapefile...")
ma = gpd.read_file(shapefile_path)
ma.crs = "EPSG:4326"

Loading Maranhão shapefile...


In [5]:
# loading the multiclass rasters
print("Loading multiclass rasters...")
multiclass_paths = sorted(glob.glob(os.path.join(data_dir, "*_multiclass.tif")))

print(f"Found {len(multiclass_paths)} multiclass rasters")

Loading multiclass rasters...
Found 24 multiclass rasters


In [6]:
# extracting the years
years = []
for path in multiclass_paths:
    fname = os.path.basename(path)
    digits = ''.join(filter(str.isdigit, fname))
    years.append(int(digits[-4:]) if len(digits) >= 4 else None)

In [7]:
# loading the arrays
lc_arrays = []
extent = None
raster_crs = None

print("Processing years:")
for path, year in zip(multiclass_paths, years):
    with rasterio.open(path) as src:
        arr = src.read(1).astype(float)
        lc_arrays.append(arr)
        
        if extent is None:
            bounds = src.bounds
            extent = (bounds.left, bounds.right, bounds.bottom, bounds.top)
            raster_crs = src.crs
    
    # calculating the areas for each class
    forest_area = np.sum(arr == 1) * 0.25
    agri_area = np.sum(arr == 2) * 0.25
    pasture_area = np.sum(arr == 3) * 0.25
    
    print(f"  {year}: Forest={forest_area:,.0f} km² | Agri={agri_area:,.0f} km² | Pasture={pasture_area:,.0f} km²")

print(f"Loaded {len(lc_arrays)} rasters")

Processing years:
  2001: Forest=60,612 km² | Agri=2,605 km² | Pasture=231,134 km²
  2002: Forest=54,437 km² | Agri=2,472 km² | Pasture=239,392 km²
  2003: Forest=49,906 km² | Agri=2,642 km² | Pasture=244,002 km²
  2004: Forest=45,960 km² | Agri=2,764 km² | Pasture=246,360 km²
  2005: Forest=44,393 km² | Agri=2,976 km² | Pasture=247,262 km²
  2006: Forest=40,937 km² | Agri=3,203 km² | Pasture=250,018 km²
  2007: Forest=39,440 km² | Agri=3,357 km² | Pasture=250,238 km²
  2008: Forest=34,968 km² | Agri=3,726 km² | Pasture=252,082 km²
  2009: Forest=34,334 km² | Agri=3,899 km² | Pasture=254,068 km²
  2010: Forest=34,740 km² | Agri=3,956 km² | Pasture=252,582 km²
  2011: Forest=36,248 km² | Agri=4,225 km² | Pasture=251,115 km²
  2012: Forest=35,394 km² | Agri=4,402 km² | Pasture=251,080 km²
  2013: Forest=32,732 km² | Agri=4,539 km² | Pasture=253,936 km²
  2014: Forest=32,719 km² | Agri=4,611 km² | Pasture=254,272 km²
  2015: Forest=33,545 km² | Agri=4,853 km² | Pasture=250,632 km²
  2016:

In [8]:
# reprojecting the shapefile if needed
if raster_crs != ma.crs:
    ma = ma.to_crs(raster_crs)

In [12]:
# creating the animation
print("Creating multiclass animation...")

# color map: 0=gray (other), 1=green (forest), 2=gold (agri), 3=tan (pasture)
cmap = ListedColormap(["#e6e6e6", "#006400", "#d4a017", "#c19a6b"])

# creating the figure
fig, ax = plt.subplots(figsize=(10, 12))
fig.patch.set_facecolor('white')

# displaying the first frame
im = ax.imshow(lc_arrays[0], cmap=cmap, vmin=0, vmax=3, extent=extent, interpolation='nearest')

title = ax.text(
    0.5, 1.07,
    f"Maranhão Land Cover — {years[0]}",
    transform=ax.transAxes,
    ha="center", fontsize=20, fontweight="bold"
)

forest_area_0 = np.sum(lc_arrays[0] == 1) * 0.25
agri_area_0 = np.sum(lc_arrays[0] == 2) * 0.25
pasture_area_0 = np.sum(lc_arrays[0] == 3) * 0.25

stats_text = ax.text(
    0.5, 1.02,
    f"Forest: {forest_area_0:,.0f} km²  |  Agriculture: {agri_area_0:,.0f} km²  |  Pasture/Shrub: {pasture_area_0:,.0f} km²",
    transform=ax.transAxes,
    ha="center", fontsize=12,
    bbox=dict(boxstyle='round', facecolor='white', alpha=0.9, edgecolor='black')
)

# drawing the boundary
ma.boundary.plot(ax=ax, color="black", linewidth=1.5)

# setting the extent
ax.set_xlim(extent[0], extent[1])
ax.set_ylim(extent[2], extent[3])
ax.set_xticks([])
ax.set_yticks([])
ax.set_aspect('equal')

# adding the legend
legend_elements = [
    Patch(facecolor='#006400', label='Forest'),
    Patch(facecolor='#d4a017', label='Agriculture'),
    Patch(facecolor='#c19a6b', label='Pasture/Shrub/Savanna'),
    Patch(facecolor='#e6e6e6', label='Other')
]
ax.legend(
    handles=legend_elements,
    loc='lower right',
    frameon=True,
    fancybox=True,
    shadow=True,
    fontsize=11
)

# updating the function
def update(i):
    """Update map and statistics"""
    # Update raster
    im.set_data(lc_arrays[i])
    
    # Update title
    title.set_text(f"Maranhão Land Cover — {years[i]}")
    
    # Update statistics
    forest_area = np.sum(lc_arrays[i] == 1) * 0.25
    agri_area = np.sum(lc_arrays[i] == 2) * 0.25
    pasture_area = np.sum(lc_arrays[i] == 3) * 0.25
    
    stats_text.set_text(
        f"Forest: {forest_area:,.0f} km²  |  Agriculture: {agri_area:,.0f} km²  |  Pasture/Shrub: {pasture_area:,.0f} km²"
    )
    
    return [im, title, stats_text]

# creating and saving the animation
print(f"\nGenerating animation with {len(lc_arrays)} frames...")

anim = FuncAnimation(
    fig,
    update,
    frames=len(lc_arrays),
    interval=interval,
    blit=True,
    repeat=True
)

print(f"Saving animation to: {output_gif}")

anim.save(output_gif, writer='pillow', fps=fps, dpi=dpi)

print(f"Output file: {output_gif}")
print(f"Duration: ~{len(lc_arrays) / fps:.1f} seconds")

plt.close()

Creating multiclass animation...

Generating animation with 24 frames...
Saving animation to: maranhao_multiclass_animation.gif
Output file: maranhao_multiclass_animation.gif
Duration: ~12.0 seconds


In [13]:
from IPython.display import HTML
HTML(anim.to_html5_video())